In [1]:
import numpy as np
import RedLionfishDeconv.RLDeconv3DScipy as rls
import tifffile
import os
import napari

In [2]:
#Path is referenced from notebook location
data = tifffile.imread("testdata/gendata_psfconv_poiss.tif")
psf = tifffile.imread("testdata/PSF_RFI_8bit.tif")

In [3]:
data.shape

(160, 160, 160)

In [4]:
psf.sum()

1094998

In [5]:
psf_f = psf.astype(np.float32)

In [6]:
psf_norm = psf_f/psf_f.sum()

In [7]:
psf_norm.sum()

1.0

In [9]:
print("Starting RL calculation using doRLDeconvolution_DL2_4()")
res0 = rls.doRLDeconvolution_DL2_4(data,psf_norm, niter=3)
print("Completed")
print(f"res0 shape:{res0.shape}, dtype: {res0.dtype}, max,min: {res0.max()},{res0.min()}")

Starting RL calculation using doRLDeconvolution_DL2_4()
Completed
res0 shape:(160, 160, 160), dtype: float32, max,min: 1293.8343505859375,0.0


In [8]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()")
res1 = rlocl.nonBlock_RLDeconvolutionReiknaOCL(data,psf_norm, niter=3)
print("Completed")
print(f"res1 shape:{res1.shape}, dtype: {res1.dtype}, max,min: {res1.max()},{res1.min()}")

Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()
Completed
res1 shape:(160, 160, 160), dtype: float32, max,min: 1293.8333740234375,0.0


In [10]:
import skimage.restoration
print("Starting RL calculation using skimage.restoration.richardson_lucy()")
res_skimage = skimage.restoration.richardson_lucy(data,psf_norm, num_iter = 3, clip=False)
print("Completed")
print(f"res_skimage shape:{res_skimage.shape}, dtype: {res_skimage.dtype}, max,min: {res_skimage.max()},{res_skimage.min()}")

Starting RL calculation using skimage.restoration.richardson_lucy()
Completed
res_skimage shape:(160, 160, 160), dtype: float64, max,min: 283.6625643370356,-1.1211678747872315e-45


Strangely they give different results in intensity, so maybe not a good way to test RL code

In [11]:
NV=napari.Viewer()
NV.add_image(res0)
NV.add_image(res1)
NV.add_image(res_skimage)

<Image layer 'res_skimage' at 0x1b345555760>

In [12]:
# absolute(a - b) <= (atol + rtol * absolute(b))

#np.allclose(res0,res1,rtol=1e-5, atol=1e-30)
#np.allclose(res0,res1,atol=1e-30)
np.allclose(res0,res1) #Use defaule rtol and atol 

True

In [13]:
#Maximum difference
np.max(np.abs(res0-res1))

0.0009765625

In [14]:
#Maximum difference
np.max(np.abs(res0-res_skimage))

1035.6352172565466

In [16]:
#Just for curiosity, compare the sum of values before and after convolution
print(np.sum(data))
print(np.sum(res0))
print(np.sum(res1))
print(np.sum(res_skimage))

18409407
18559448.0
18559448.0
18626902.969301894


## Large(r) data

In [17]:
#Generate 256x256x256 data for testing
#Create a 2d array to do FFT
ashape = (256,256,256)

a = np.zeros(ashape, dtype=np.float32)

#Add a few cubes in grid-like locations
cubesize=2
cubespacing=60
for iz in range(int(cubespacing/2),ashape[0],cubespacing):
    for iy in range(int(cubespacing/2),ashape[1],cubespacing):
        for ix in range(int(cubespacing/2),ashape[2],cubespacing):
            a[iz:iz+cubesize , iy:iy+cubesize , ix:ix+cubesize] = np.ones((cubesize,cubesize,cubesize))

#Convolute with experimental PSF
psf = tifffile.imread("testdata/PSF_RFI_8bit.tif")
psf_f = psf.astype(np.float32)
psf_norm = psf_f/ np.sum(psf_f)
assert np.sum(psf_norm)==1.0

import scipy.signal
data_large = scipy.signal.convolve(a, psf_norm, mode='same')
print(data_large.shape)

(256, 256, 256)


In [18]:
print("Starting RL calculation using doRLDeconvolution_DL2_4()")
res0 = rls.doRLDeconvolution_DL2_4(data_large,psf_norm, niter=3)
print("Completed")
print(f"res0 shape:{res0.shape}, dtype: {res0.dtype}, max,min: {res0.max()},{res0.min()}")

Starting RL calculation using doRLDeconvolution_DL2_4()
Completed
res0 shape:(256, 256, 256), dtype: float32, max,min: 0.009696509689092636,-0.00010524897516006604


In [19]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()")
res1 = rlocl.nonBlock_RLDeconvolutionReiknaOCL(data_large,psf_norm, niter=3)
print("Completed")
print(f"res1 shape:{res1.shape}, dtype: {res1.dtype}, max,min: {res1.max()},{res1.min()}")

Starting RL calculation using nonBlock_RLDeconvolutionReiknaOCL()
Completed
res1 shape:(256, 256, 256), dtype: float32, max,min: 0.00969840306788683,-3.860086144413799e-05


In [25]:
import RedLionfishDeconv.RLDeconv3DReiknaOCL as rlocl
print("Starting RL calculation using block_RLDeconv3DReiknaOCL()")
res2 = rlocl.block_RLDeconv3DReiknaOCL(data_large,psf_norm,niter=3, max_dim_size = 128)
print("Completed")
print(f"res2 shape:{res2.shape}, dtype: {res2.dtype}, max,min: {res2.max()},{res2.min()}")

Starting RL calculation using block_RLDeconv3DReiknaOCL()
Completed
res2 shape:(256, 256, 256), dtype: float32, max,min: 0.00956675037741661,-0.000715897127520293


In [20]:
import skimage.restoration
print("Starting RL calculation using skimage.restoration.richardson_lucy()")
res_skimage = skimage.restoration.richardson_lucy(data_large,psf_norm, num_iter = 3, clip=False)
print("Completed")
print(f"res_skimage shape:{res_skimage.shape}, dtype: {res_skimage.dtype}, max,min: {res_skimage.max()},{res_skimage.min()}")

Starting RL calculation using skimage.restoration.richardson_lucy()
Completed
res_skimage shape:(256, 256, 256), dtype: float64, max,min: 0.002362383784275899,-7.524207186831826e-13


In [53]:
print("Starting RL calculation using doRLDeconvolution12()")
res3 = rls.doRLDeconvolution12(data_large,psf_norm, niter=3)
print("Completed")
print(f"res3 shape:{res3.shape}, dtype: {res3.dtype}, max,min: {res3.max()},{res3.min()}")

Starting RL calculation using doRLDeconvolution12()
Completed
res3 shape:(256, 256, 256), dtype: uint8, max,min: 255,0


In [51]:
#Maximum difference
print(np.max(np.abs(res0-res1)))
print(np.max(np.abs(res0-res2)))
print(np.max(np.abs(res0-res2)))

0.00027373407
0.0013149539


In [27]:
#Just for curiosity, compare the sum of values before and after convolution
print(np.sum(data_large))
print(np.sum(res0))
print(np.sum(res1))
print(np.sum(res2))
print(np.sum(res_skimage))

512.0000046125008
516.3106
516.30084
517.5604
517.5152769736716


In [31]:
print(np.max(data_large), np.min(data_large))
print(np.max(res0), np.min(res0))
print(np.max(res1), np.min(res1))
print(np.max(res2), np.min(res2))
print(np.max(res_skimage), np.min(res_skimage))

0.0017753456515616454 -8.283871215872297e-11
0.00969651 -0.000105248975
0.009698403 -3.860086e-05
0.00956675 -0.0007158971
0.002362383784275899 -7.524207186831826e-13


In [32]:
contrastlimit0 = [0.0,0.01]
NV=napari.Viewer()
NV.add_image(data_large, contrast_limits=contrastlimit0)
NV.add_image(res0, contrast_limits=contrastlimit0)
NV.add_image(res1, contrast_limits=contrastlimit0)
NV.add_image(res2, contrast_limits=contrastlimit0)
NV.add_image(res_skimage, contrast_limits=contrastlimit0)

<Image layer 'res_skimage' at 0x18eb03aefa0>

In [50]:
#Maximum difference
print(np.max(np.abs(res1-res0)))
print(np.max(np.abs(res2-res0)))
print(np.max(np.abs(res2-res1)))
print(np.max(np.abs(res_skimage-res0)))

0.00027373407
0.0013149539
0.0013279477
0.007578507633556988


In [49]:
#Check between each others
print(np.allclose(res1,res0, rtol=1e-2, atol=1e-2))
print(np.allclose(res2,res0, rtol=1e-2, atol=1e-2))
print(np.allclose(res2,res1, rtol=1e-2, atol=1e-2))
print(np.allclose(res_skimage,res0, rtol=1e-2, atol=1e-2))

True
True
True
True
